In [41]:
# get ireland weather stations info
# get EFI/CDF and wind gusts EFI for stations
# note it takes a while for the API requests to go through

stations_url = 'https://www.met.ie/climate/weather-observing-stations'

import requests
req = requests.get(stations_url)
if req.ok:
    text = req.text
req.close()


In [42]:
import re
from bs4 import BeautifulSoup
regex = r"""class="collapsed">(.*?)</a>.*?<p><strong>Location</strong><br />(.*?)<br />(.*?)<br />(.*?)m above mean sea level<"""
#regex = r"""class="collapsed">(.*?)</a>.*?<p>"""
res = re.findall(regex, text, flags=re.DOTALL|re.I)

In [43]:
# check res for debugging
# res

In [44]:
def convert_coord_to_decimal(coord):
    [h, m, s, hemisphere] = coord
    matchW = re.search(r'W', hemisphere, flags=re.IGNORECASE)
    matchS = re.search(r'S', hemisphere, flags=re.IGNORECASE)
    # multiply decimal coords by 1 or -1 depending on which hemisphere
    if matchW or matchS:
        positive = -1.0
    else:
        positive = 1.0
    return positive * (
        float(h) + 
        (float(m) / 60.0) +
        (float(s) / 3600.0)
    )

def parse_string_to_coords_hemisphere(str):
    matches = re.findall(r'(\d+)[^\d]+(\d+)[^\d]+(\d+)[^\d]*([NEWS])', str, flags=re.IGNORECASE)
    return matches[0]

def convert_str_to_decimal(str):
    return convert_coord_to_decimal(parse_string_to_coords_hemisphere(str))

print("Location, Lat, Long, Altitude (m)\n")
ireland_weather_locations = {}
for [location, lat, long, alt] in res:
    location = BeautifulSoup(location).get_text().strip()
    print(location)
    lat = BeautifulSoup(lat).get_text().strip()
    long = BeautifulSoup(long).get_text().strip()
    lat_dec = convert_str_to_decimal(lat)
    long_dec = convert_str_to_decimal(long)
    print(f"{lat_dec:.4f}")
    print(f"{long_dec:.4f}")
    alt = BeautifulSoup(alt).get_text().strip()
    print(alt)
    print("")
    ireland_weather_locations[location] = [lat_dec, long_dec, alt]
    

Location, Lat, Long, Altitude (m)

Athenry
53.2892
-8.7856
40

Ballyhaise
54.0514
-7.3097
78

Belmullet
54.2275
-10.0069
9

Carlow Oakpark
52.8611
-6.9153
62

Claremorris
53.7108
-8.9925
68

Dunsany
53.5158
-6.6600
83

Fermoy Moorepark
52.1639
-8.2639
46

Finner
54.4939
-8.2431
33

Gurteen
53.0531
-8.0086
75

Johnstown Castle
52.2978
-6.4967
62

Mace Head
53.3258
-9.9008
21

Malin Head
55.3722
-7.3389
22

Markree
54.1750
-8.4556
34

Mount Dillon
53.7269
-7.9808
39

Mullingar
53.5372
-7.3622
101

Newport
53.9222
-9.5722
22

Phoenix Park
53.3639
-6.3333
48

Roches Point
51.7931
-8.2444
43

Sherkin Island
51.4764
-9.4278
21

Valentia
51.9397
-10.2444
25



In [45]:

api_url = "https://charts.ecmwf.int/opencharts-api/v1/"
print('{}products/'.format(api_url))
import requests

result =requests.get('{}products/'.format(api_url))

all_data = result.json()

https://charts.ecmwf.int/opencharts-api/v1/products/


In [46]:
import json
d = json.dumps(all_data, indent=4)
#print(d)

In [47]:
# wind gusts
openchart_efi_gusts = 'efi2web_10fg'
schema_url_openchart_efi_gusts = [x['schema-url'] for x in all_data if x['name'] == openchart_efi_gusts][0]
product_url_openchart_efi_gusts = [x['product-url'] for x in all_data if x['name'] == openchart_efi_gusts][0]
result = requests.get(schema_url_openchart_efi_gusts)
schema_openchart_efi_gusts = result.json()
d = json.dumps(schema_openchart_efi_gusts, indent=4)
print(d)

{
    "openapi": "3.0.2",
    "info": {
        "version": "1.0",
        "title": "Export Opencharts"
    },
    "servers": [
        {
            "url": "https://charts.ecmwf.int/opencharts-api/v1/"
        }
    ],
    "paths": {
        "/products": {
            "get": {
                "responses": {
                    "200": {
                        "content": {
                            "application/json": {
                                "schema": {
                                    "$ref": "#/components/schemas/products-list"
                                }
                            }
                        },
                        "description": "Data are ready to be downloaded"
                    }
                }
            }
        },
        "/products/efi2web_10fg/": {
            "get": {
                "responses": {
                    "default": {
                        "content": {
                            "application/json": {
            

In [48]:
# efi/cdf
openchart_efi_cdf = 'opencharts_efi-cdf-meteogram'
schema_url_openchart_efi_cdf = [x['schema-url'] for x in all_data if x['name'] == openchart_efi_cdf][0]
product_url_openchart_efi_cdf = [x['product-url'] for x in all_data if x['name'] == openchart_efi_cdf][0]
result = requests.get(schema_url_openchart_efi_cdf)
schema_openchart_efi_cdf = result.json()
d = json.dumps(schema_openchart_efi_cdf, indent=4)
#print(d)

In [49]:
print("Parameters for EFI GUSTS")
parameters_efi_gusts = schema_openchart_efi_gusts['paths']['/products/' + openchart_efi_gusts + '/']['get']['parameters']
parameter_names_efi_gusts = [x['name'] for x in parameters_efi_gusts]
print(parameter_names_efi_gusts)
print("")
print("Parameter schema:")
#print(json.dumps(parameters_efi_gusts,indent=4))

Parameters for EFI GUSTS
['format', 'base_time', 'quantile', 'day', 'area']

Parameter schema:


In [50]:
print("Parameters for EFI CDF")
parameters_efi_cdf = schema_openchart_efi_cdf['paths']['/products/' + openchart_efi_cdf + '/']['get']['parameters']
parameter_efi_cdf_names = [x['name'] for x in parameters_efi_cdf]
print(parameter_efi_cdf_names)
print("")
print("Parameter schema:")
#print(json.dumps(parameters_efi_cdf,indent=4))

Parameters for EFI CDF
['format', 'base_time', 'step', 'valid_time', 'city', 'altitude', 'lon', 'lat']

Parameter schema:


In [51]:
print("API urls")
print(product_url_openchart_efi_cdf)
print(product_url_openchart_efi_gusts)

API urls
https://charts.ecmwf.int/opencharts-api/v1/products/opencharts_efi-cdf-meteogram/
https://charts.ecmwf.int/opencharts-api/v1/products/efi2web_10fg/


In [52]:
from IPython.display import display, HTML

def get_EFI_gusts(day, area):
    params = {'day': day, 'area': area}
    result = requests.get(product_url_openchart_efi_gusts, params=params)
    if result.status_code == 200:
        data = result.json()
        url = data['data']['link']['href']
        return url
    else:
        return None

def get_EFI_CDF(valid_time, lat, lon, altitude=None):
    if altitude is None:
        params = {'valid_time': valid_time, 'lat': lat, 'lon': lon}
    else:
        params = {'valid_time': valid_time, 'lat': lat, 'lon': lon, 'altitude': altitude}
    result = requests.get(product_url_openchart_efi_cdf, params=params)
    if result.status_code == 200:
        data = result.json()
        url = data['data']['link']['href']
        return url
    else:
        return None

In [53]:
days = list(range(1,8))
# EFI gusts only has days 1-7
#days.append("7-9")
#days.append("10-15")
area = "North West Europe"

from IPython.display import HTML, display

# for sequentially displaying
"""
print("ECMWF EFI gusts for NW Europe\n")
html_code = ""
for day in days:
    url = get_EFI_gusts(day, area)
    html_code += f"<p>ECMWF EFI gusts for day {day} \n</p><img src='{url}'/>"

display(HTML(html_code))
"""

image_paths_gusts = []
for day in days:
    url = get_EFI_gusts(day, area)
    if url:
        image_paths_gusts.append(url)
    else:
        print(f"Error retrieving data for day {day}")

In [54]:
# Create HTML code for the image slider and buttons
html_code = """
<div id="imageGusts-controls">
    <button onclick="previousGustsImage()">Previous</button>
    <input type="range" id="imageGusts-slider" min="0" value="0" onchange="updateGustsImage()">
    <button onclick="nextGustsImage()">Next</button>
</div>

<div id="imageGusts-container">
    <img src="" id="displayedGusts-image" style="width: 100%;">
</div>

<script>
    var currentImageGustsIndex = 0;
    var imageGustsPaths = """ + str(image_paths_gusts) + """;
    var displayedGustsImage = document.getElementById("displayedGusts-image");
    var imageGustsSlider = document.getElementById("imageGusts-slider");

    function displayImageGusts(index) {
        var imageGusts_url = imageGustsPaths[index];
        displayedGustsImage.src = imageGusts_url;
        currentImageGustsIndex = index;
        imageGustsSlider.max = imageGustsPaths.length - 1;
        imageGustsSlider.value = index;
    }

    function previousGustsImage() {
        currentImageGustsIndex = (currentImageGustsIndex - 1 + imageGustsPaths.length) % imageGustsPaths.length;
        displayImage(currentImageGustsIndex);
    }

    function nextGustsImage() {
        currentImageGustsIndex = (currentImageGustsIndex + 1) % imageGustsPaths.length;
        displayImageGusts(currentImageGustsIndex);
    }

    function updateGustsImage() {
        currentImageGustsIndex = parseInt(imageGustsSlider.value);
        displayImageGusts(currentImageGustsIndex);
    }

    // Initial display
    displayImageGusts(currentImageGustsIndex);
</script>
"""

# Display the initial image and slider
display(HTML(html_code))

In [55]:
valid_time = "2023-09-28T00:00:00Z"


from IPython.display import HTML, display
# for sequentially displaying
"""
print("ECMWF EFI/CDF for Ireland Weather Stations @ https://www.met.ie")
print(f"For end time: {valid_time}\n")
html_code = ""
for station in ireland_weather_locations:
    [lat, lon, altitude] = ireland_weather_locations[station]
    url = get_EFI_CDF(valid_time, lat, lon, altitude)
    html_code += f"<p>ECMWF EFI for {station} \n</p><img src='{url}'/>"

display(HTML(html_code))
"""

image_paths_cdf = []
image_captions_cdf = []
for station in ireland_weather_locations:
    [lat, lon, altitude] = ireland_weather_locations[station]
    url = get_EFI_CDF(valid_time, lat, lon, altitude)
    if url:
        image_paths_cdf.append(url)
        image_captions_cdf.append(f"ECMWF EFI for {station}\nFor end time: {valid_time}")
    else:
        print(f"Error retrieving data for station: {station}")

In [56]:
# Create HTML code for the image slider and buttons
html_code_cdf = """
<div id="CDF-mage-controls">
    <button onclick="CDFpreviousImage()">Previous</button>
    <input type="range" id="CDF-image-slider" min="0" value="0" onchange="CDFupdateImage()">
    <button onclick="CDFnextImage()">Next</button>
</div>

<div id="CDF-image-caption">
    <p>Caption for Image 1</p>
</div>

<div id="CDF-image-container">
    <img src="" id="CDF-displayed-image" style="width: 100%;">
</div>

<script>
    var currentImageIndexCDF = 0;
    var imagePathsCDF = """ + str(image_paths_cdf) + """;
    var imageCaptionsCDF = """ + str(image_captions_cdf) + """;
    var displayedImageCDF = document.getElementById("CDF-displayed-image");
    var imageSliderCDF = document.getElementById("CDF-image-slider");
    var imageCaptionCDF = document.getElementById("CDF-image-caption");

    function CDFdisplayImageAndCaption(index) {
        console.log(imagePathsCDF)
        var image_url = imagePathsCDF[index];
        var caption = imageCaptionsCDF[index];
        displayedImageCDF.src = image_url;
        imageCaptionCDF.innerHTML = "<p>" + caption + "</p>";
        currentImageIndexCDF = index;
        imageSliderCDF.max = imagePathsCDF.length - 1;
        imageSliderCDF.value = index;
    }

    function CDFpreviousImage() {
        currentImageIndexCDF = (currentImageIndexCDF - 1 + imagePathsCDF.length) % imagePathsCDF.length;
        CDFdisplayImageAndCaption(currentImageIndexCDF);
    }

    function CDFnextImage() {
        currentImageIndexCDF = (currentImageIndexCDF + 1) % imagePathsCDF.length;
        CDFdisplayImageAndCaption(currentImageIndexCDF);
    }

    function CDFupdateImage() {
        currentImageIndexCDF = parseInt(imageSliderCDF.value);
        CDFdisplayImageAndCaption(currentImageIndexCDF);
    }

    // Initial display
    CDFdisplayImageAndCaption(currentImageIndexCDF);
</script>
"""

# Display the initial image, caption, and slider
display(HTML(html_code_cdf))

In [166]:
# UKV (2km) model
# https://metoffice.apiconnect.ibmcloud.com/metoffice/production/

# max 1 hour gusts from 8am to 8pm UTC
# checking also to see if it exceeds 150km/h threshold (41.67 m/s)

weather_stations = ireland_weather_locations
gust_threshold = 41.67

import pygrib
import math
from datetime import datetime, timedelta

# Define your base time (please replace with the actual base time)
base_time = datetime(2023, 9, 26, 9, 0, 0)

# Load the GRIB file
gusts_grib_file = "agl_wind-speed-gust-max-1h_10.0_2023092609.grib"
terrain_height_grib_file = "ground_model-terrain-height_2023092609.grib"

# Function to calculate the distance between two sets of coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    lat_diff = lat1 - lat2
    lon_diff = lon1 - lon2
    return math.sqrt((lat_diff * 60) ** 2 + (lon_diff * 60) ** 2)  # Convert to nautical miles

# Function to find the nearest point in the GRIB file
def find_nearest_point(grb, target_lat, target_lon):
    latitudes, longitudes = grb.latlons()
    min_distance = None
    nearest_lat = None
    nearest_lon = None
    for i in range(len(latitudes)):
        for j in range(len(longitudes)):
            if j >= len(latitudes[i]):
                break
            cur_lat = latitudes[i][j]
            cur_lon = longitudes[i][j]
            distance = calculate_distance(target_lat, target_lon, cur_lat, cur_lon)

            if min_distance is None or distance < min_distance:
                min_distance = distance
                nearest_lat = cur_lat
                nearest_lon = cur_lon

    return nearest_lat, nearest_lon

# Open the GRIB file
gusts_grbs = pygrib.open(gusts_grib_file)
terrain_heights_grbs = pygrib.open(gusts_grib_file)

# Initialize variables to store the maximum gust information
all_max_max_gust = None
all_max_gust_station = None
all_max_gust_time = None
all_max_gust_distance = None
all_units = None
all_max_gust_station_altitude = None
all_max_gust_terrain_height = None

max_max_gusts_by_station = {}
num_stations_exceeding_threshold = 0
terrain_height_units = None
# Loop through each weather station
for station, (lat, lon, altitude) in weather_stations.items():
    # rewind since we are looping multiple times
    gusts_grbs.rewind()
    max_max_gust = None
    max_gust_station = None
    max_gust_time = None
    max_gust_distance = None
    max_nearest_lat = None
    max_nearest_lon = None
    units = None
    for grb in gusts_grbs:
        units = grb.parameterUnits
        # print(f"Checking {base_time + timedelta(hours=int(grb.step))}")
        if grb.parameterName == 'Wind speed (gust)':
            nearest_lat, nearest_lon = find_nearest_point(grb, lat, lon)
            distance = calculate_distance(lat, lon, nearest_lat, nearest_lon)
            point_max_gust = grb.data(lat1 = nearest_lat, lon1 = nearest_lon, lat2 = nearest_lat, lon2 = nearest_lon)[0][0]
            # Check if this gust is greater than the current maximum
            if max_max_gust is None or point_max_gust > max_max_gust:
                max_max_gust = point_max_gust
                max_gust_station = station
                max_gust_time = base_time + timedelta(hours=int(grb.step))
                max_gust_distance = distance
                max_nearest_lat = nearest_lat
                max_nearest_lon = nearest_lon

    # Print the result
    if max_max_gust is not None:
        if max_max_gust > gust_threshold:
            num_stations_exceeding_threshold += 1
        
        terrain_heights_grbs.rewind()
        # get altitude differential
        terrain_height = None
        for terrain_grb in terrain_heights_grbs:
            # in case there is some difference between grid points
            nearest_lat, nearest_lon = find_nearest_point(terrain_grb, nearest_lat, nearest_lon)
            terrain_height = terrain_grb.data(lat1 = nearest_lat, lon1 = nearest_lon, lat2 = nearest_lat, lon2 = nearest_lon)[0][0]
            break
        
        max_max_gusts_by_station[max_gust_station] = {
            'valid_time': max_gust_time,
            'max_max_gust': max_max_gust,
            'distance': max_gust_distance,
            'terrain_height': terrain_height
        }
        
        print(f"Weather Station: {max_gust_station}")
        print(f"  Valid Time: {max_gust_time}")
        print(f"  Max Wind Gust Value (10m): {max_max_gust:0.1f} {units}")
        print(f"  Distance from model (nautical miles): {max_gust_distance:0.1f}")
        print(f"  Station altitude (official): {altitude} m")
        if terrain_height:
            print(f"  Terrain height (model): {terrain_height:.1f} m")
            model_altitude_differential = terrain_height - float(altitude)
            print(f"  Model height differential: {model_altitude_differential:.1f} m")
        if all_max_max_gust is None or max_max_gust > all_max_max_gust:
            all_max_max_gust = max_max_gust
            all_max_gust_station = max_gust_station
            all_max_gust_time = max_gust_time
            all_max_gust_distance = max_gust_distance
            all_max_gust_terrain_height = terrain_height
            all_max_gust_station_altitude = altitude
            all_units = units
    else:
        print("   No data found in the GRIB file.")

print('--------------')
print('Station with highest max value')
if max_gust is None or point_max_gust > max_max_gust:
    print(f"Weather Station: {all_max_gust_station}")
    print(f"  Valid Time: {all_max_gust_time}")
    print(f"  Max Wind Gust Value (10m): {all_max_max_gust:0.1f} {all_units}")
    print(f"  Distance from model (nautical miles): {all_max_gust_distance:0.1f}")
    print(f"  Station altitude (official): {all_max_gust_station_altitude} m")
    if all_max_gust_terrain_height:
        print(f"  Terrain height (model): {all_max_gust_terrain_height:0.1f} m")
        model_altitude_differential = all_max_gust_terrain_height + float(all_max_gust_station_altitude)
        print(f"  Model height differential: {model_altitude_differential:.1f} m")
print("")
print(f"Number of stations exceeding threshold of {gust_threshold} {all_units}: {num_stations_exceeding_threshold}")
# Close the GRIB file
gusts_grbs.close()
terrain_heights_grbs.close()

Weather Station: Athenry
  Valid Time: 2023-09-27 17:00:00
  Max Wind Gust Value (10m): 31.4 m/s
  Distance from model (nautical miles): 0.6
  Station altitude (official): 40 m
  Terrain height (model): 16.8 m
  Model height differential: -23.2 m
Weather Station: Ballyhaise
  Valid Time: 2023-09-27 18:00:00
  Max Wind Gust Value (10m): 26.2 m/s
  Distance from model (nautical miles): 0.7
  Station altitude (official): 78 m
  Terrain height (model): 14.4 m
  Model height differential: -63.6 m
Weather Station: Belmullet
  Valid Time: 2023-09-27 11:00:00
  Max Wind Gust Value (10m): 23.2 m/s
  Distance from model (nautical miles): 0.6
  Station altitude (official): 9 m
  Terrain height (model): 14.4 m
  Model height differential: 5.4 m
Weather Station: Carlow Oakpark
  Valid Time: 2023-09-27 13:00:00
  Max Wind Gust Value (10m): 31.9 m/s
  Distance from model (nautical miles): 0.7
  Station altitude (official): 62 m
  Terrain height (model): 16.7 m
  Model height differential: -45.3 m
Wea

In [168]:
import numpy as np

# http://www.webmet.com/met_monitoring/625.HTML
# wind profiles are for wind speed (mean) not gusts and using wind profiles (logarthmic adjustment)
# for this method is not directly applicable....
# this is just a hint towards sensitivity of the models for meeting the threshold

p_values = np.arange(0.1,0.7,0.1)
for p in p_values:
    for station, max_data in max_max_gusts_by_station.items():
        station_altitude = 2 + float(weather_stations[station][2])
        reference_altitude = 10 + max_data['terrain_height']
        model_max_gust = max_data['max_max_gust']

        adj_max_gust = model_max_gust * np.power(station_altitude / reference_altitude, p)
        if adj_max_gust > gust_threshold:
            print(f"p={p}, {station} Adjusted gust: {adj_max_gust:0.1f} m/s")

p=0.4, Ballyhaise Adjusted gust: 42.2 m/s
p=0.4, Carlow Oakpark Adjusted gust: 45.3 m/s
p=0.4, Dunsany Adjusted gust: 42.5 m/s
p=0.4, Gurteen Adjusted gust: 43.6 m/s
p=0.4, Johnstown Castle Adjusted gust: 44.2 m/s
p=0.4, Mullingar Adjusted gust: 45.8 m/s
p=0.5, Ballyhaise Adjusted gust: 47.5 m/s
p=0.5, Carlow Oakpark Adjusted gust: 49.4 m/s
p=0.5, Dunsany Adjusted gust: 48.2 m/s
p=0.5, Gurteen Adjusted gust: 48.4 m/s
p=0.5, Johnstown Castle Adjusted gust: 47.9 m/s
p=0.5, Mullingar Adjusted gust: 52.6 m/s
p=0.6, Ballyhaise Adjusted gust: 53.5 m/s
p=0.6, Carlow Oakpark Adjusted gust: 53.9 m/s
p=0.6, Claremorris Adjusted gust: 45.1 m/s
p=0.6, Dunsany Adjusted gust: 54.6 m/s
p=0.6, Gurteen Adjusted gust: 53.7 m/s
p=0.6, Johnstown Castle Adjusted gust: 51.9 m/s
p=0.6, Mullingar Adjusted gust: 60.5 m/s
